## Tugas Praktikum

Tugas 1: Multiple Linear Regression<br>

1. Identifikasi variabel-variabel yang akan digunakan sebagai variabel bebas (fitur) dan variabel target (biaya medis personal).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('/content/insurance (1).csv')
data.head()

# melihat beberapa data awal
data.head()

# mengecek ukuran data
data.shape

# informasi tentang data
data.info()

# deskripsi data
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [ ]:
# Cek missing values
print("Missing Values:")
print(data.isnull().sum())
print("\n" + "="*50 + "\n")

# Cek nilai unik pada kolom kategorikal
print("Nilai unik pada kolom kategorikal:")
print(f"Sex: {data['sex'].unique()}")
print(f"Smoker: {data['smoker'].unique()}")
print(f"Region: {data['region'].unique()}")

Missing Values:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


Nilai unik pada kolom kategorikal:
Sex: ['female' 'male']
Smoker: ['yes' 'no']
Region: ['southwest' 'southeast' 'northwest' 'northeast']


In [ ]:
# Preprocessing: Encode variabel kategorikal
from sklearn.preprocessing import LabelEncoder

# --- Encoding ---
data_encoded = data.copy()

label_encoder_sex = LabelEncoder()
label_encoder_smoker = LabelEncoder()
label_encoder_region = LabelEncoder()

data_encoded['sex'] = label_encoder_sex.fit_transform(data_encoded['sex'])
data_encoded['smoker'] = label_encoder_smoker.fit_transform(data_encoded['smoker'])
data_encoded['region'] = label_encoder_region.fit_transform(data_encoded['region'])

print("Data setelah encoding:")
print(data_encoded.head())

X = data_encoded.drop(columns=['charges'])
y = data_encoded['charges']

# --- Split data 80:20 ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Ukuran data latih:", X_train.shape)
print("Ukuran data uji:", X_test.shape)

Data setelah encoding:
   age  sex     bmi  children  smoker  region      charges
0   19    0  27.900         0       1       3  16884.92400
1   18    1  33.770         1       0       2   1725.55230
2   28    1  33.000         3       0       2   4449.46200
3   33    1  22.705         0       0       1  21984.47061
4   32    1  28.880         0       0       1   3866.85520
Ukuran data latih: (1070, 6)
Ukuran data uji: (268, 6)


| Variabel   | Tipe Data | Deskripsi                                                           |
| ---------- | --------- | ------------------------------------------------------------------- |
| `age`      | int       | Usia individu                                                       |
| `sex`      | object    | Jenis kelamin (male/female)                                         |
| `bmi`      | float     | Body Mass Index — indikator berat badan terhadap tinggi badan       |
| `children` | int       | Jumlah anak yang ditanggung oleh asuransi                           |
| `smoker`   | object    | Status perokok (yes/no)                                             |
| `region`   | object    | Wilayah tempat tinggal (northeast, northwest, southeast, southwest) |
| `charges`  | float     | Biaya medis pribadi yang dibebankan (target/variabel dependen)      |


Identifikasi Variabel:

Variabel Bebas (X / fitur): age, sex, bmi, children, smoker, region

Variabel Target (Y): charges

2. Bagi dataset menjadi data latih (train) dan data uji (test) dengan proporsi yang sesuai.



In [ ]:
from sklearn.model_selection import train_test_split

# Split data dengan proporsi 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Ukuran data latih (X_train):", X_train.shape)
print("Ukuran data uji (X_test):", X_test.shape)
print("Ukuran target latih (y_train):", y_train.shape)
print("Ukuran target uji (y_test):", y_test.shape)

Ukuran data latih (X_train): (1070, 6)
Ukuran data uji (X_test): (268, 6)
Ukuran target latih (y_train): (1070,)
Ukuran target uji (y_test): (268,)


Korelasi Awal (Numerik)
Dari heatmap korelasi (gambar di atas), tampak bahwa:
  - age dan bmi memiliki korelasi positif dengan charges.
  - children hanya memiliki korelasi lemah terhadap charges.

| Metrik                             | Nilai         |
| ---------------------------------- | ------------- |
| **MSE (Mean Squared Error)**       | 33,596,915.85 |
| **RMSE (Root Mean Squared Error)** | 5,796.28      |
| **R² (Koefisien Determinasi)**     | 0.7836        |

Nilai R² = 0.78 berarti model mampu menjelaskan sekitar 78% variasi biaya medis berdasarkan fitur-fitur input.

Nilai RMSE ≈ 5796 menunjukkan rata-rata kesalahan prediksi sekitar $5796, yang masih cukup besar — menandakan adanya variabilitas yang tidak dijelaskan oleh model linier (mungkin hubungan non-linear, terutama efek “smoker”).

3. Lakukan feature scaling jika diperlukan.



In [ ]:
# Scaling fitur menggunakan StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data sebelum scaling (5 baris pertama X_train):")
print(X_train.head())
print("\n" + "="*50)
print("\nData setelah scaling (5 baris pertama X_train_scaled):")
print(pd.DataFrame(X_train_scaled[:5], columns=X.columns))

Data sebelum scaling (5 baris pertama X_train):
      age  sex    bmi  children  smoker  region
560    46    0  19.95         2       0       1
1285   47    0  24.32         0       0       0
1142   52    0  24.86         0       0       2
969    39    0  34.32         5       0       2
486    54    0  21.47         3       0       1


Data setelah scaling (5 baris pertama X_train_scaled):
        age       sex       bmi  children    smoker    region
0  0.472227 -1.024602 -1.756525  0.734336 -0.508747 -0.456116
1  0.543313 -1.024602 -1.033082 -0.911192 -0.508747 -1.353256
2  0.898745 -1.024602 -0.943687 -0.911192 -0.508747  0.441024
3 -0.025379 -1.024602  0.622393  3.202629 -0.508747  0.441024
4  1.040918 -1.024602 -1.504893  1.557100 -0.508747 -0.456116


4. Buat model multiple linear regression menggunakan Scikit-Learn.



In [ ]:
from sklearn.linear_model import LinearRegression

# Inisialisasi model
mlr_model = LinearRegression()

# Tampilkan informasi model
print("Model Multiple Linear Regression telah diinisialisasi")
print(f"Model: {mlr_model}")

Model Multiple Linear Regression telah diinisialisasi
Model: LinearRegression()


5. Latih model pada data latih dan lakukan prediksi pada data uji.


In [ ]:
# Latih model pada data latih
mlr_model.fit(X_train_scaled, y_train)

# Lakukan prediksi pada data uji
y_pred = mlr_model.predict(X_test_scaled)

print("Model telah dilatih!")
print("\nKoefisien model (weights) untuk setiap fitur:")
for feature, coef in zip(X.columns, mlr_model.coef_):
    print(f"  {feature}: {coef:.2f}")
print(f"\nIntercept (bias): {mlr_model.intercept_:.2f}")

# Tampilkan beberapa prediksi vs nilai aktual
print("\n" + "="*50)
print("\nPerbandingan Prediksi vs Nilai Aktual (10 data pertama):")
comparison_df = pd.DataFrame({
    'Aktual': y_test[:10].values,
    'Prediksi': y_pred[:10],
    'Selisih': y_test[:10].values - y_pred[:10]
})
print(comparison_df)

Model telah dilatih!

Koefisien model (weights) untuk setiap fitur:
  age: 3616.11
  sex: -9.39
  bmi: 2028.31
  children: 516.66
  smoker: 9557.14
  region: -302.39

Intercept (bias): 13346.09


Perbandingan Prediksi vs Nilai Aktual (10 data pertama):
        Aktual      Prediksi      Selisih
0   9095.06825   8924.407244   170.661006
1   5272.17580   7116.295018 -1844.119218
2  29330.98315  36909.013521 -7578.030371
3   9301.89355   9507.874691  -205.981141
4  33750.29180  27013.350008  6736.941792
5   4536.25900  10790.779562 -6254.520562
6   2117.33885    226.298446  1891.040404
7  14210.53595  16942.715999 -2732.180049
8   3732.62510   1056.630794  2675.994306
9  10264.44210  11267.919973 -1003.477873



6. Evaluasi model dengan menghitung metrik seperti R-squared, MSE, dan MAE. Tampilkan hasil evaluasi.

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math

# Hitung metrik evaluasi
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

# Hitung juga untuk data training untuk melihat overfitting
y_train_pred = mlr_model.predict(X_train_scaled)
r2_train = r2_score(y_train, y_train_pred)

print("="*60)
print("HASIL EVALUASI MODEL MULTIPLE LINEAR REGRESSION")
print("="*60)
print(f"\nPerforma pada Data Training:")
print(f"  R-squared (R²): {r2_train:.4f}")

print(f"\nPerforma pada Data Testing:")
print(f"  R-squared (R²): {r2:.4f}")
print(f"  Mean Squared Error (MSE): {mse:.2f}")
print(f"  Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"  Mean Absolute Error (MAE): {mae:.2f}")

print("\n" + "="*60)
print("INTERPRETASI METRIK:")
print("="*60)
print(f"• R² = {r2:.4f} berarti model dapat menjelaskan {r2*100:.2f}% variasi dalam data")
print(f"• RMSE = ${rmse:.2f} adalah rata-rata kesalahan prediksi dalam dollar")
print(f"• MAE = ${mae:.2f} adalah rata-rata selisih absolut prediksi dengan nilai aktual")

HASIL EVALUASI MODEL MULTIPLE LINEAR REGRESSION

Performa pada Data Training:
  R-squared (R²): 0.7417

Performa pada Data Testing:
  R-squared (R²): 0.7833
  Mean Squared Error (MSE): 33635210.43
  Root Mean Squared Error (RMSE): 5799.59
  Mean Absolute Error (MAE): 4186.51

INTERPRETASI METRIK:
• R² = 0.7833 berarti model dapat menjelaskan 78.33% variasi dalam data
• RMSE = $5799.59 adalah rata-rata kesalahan prediksi dalam dollar
• MAE = $4186.51 adalah rata-rata selisih absolut prediksi dengan nilai aktual


Catatan Tambahan:<br>
- Pastikan untuk memberikan penjelasan singkat tentang dataset yang digunakan, termasuk deskripsi variabel-variabel yang relevan.

- Tampilkan visualisasi data yang relevan seperti scatter plot atau grafik lainnya jika diperlukan untuk pemahaman yang lebih baik.

- Analisis hasil yang Anda dapatkan dalam tugas praktikum Anda.
